## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [309]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        if codigo.isdigit() and len(codigo) == 4:
            self.codigo = codigo
        else:
            raise ValueError("4 dígitos minimo")

        if isinstance(nombre, str) and 1 <= len(nombre) <= 100:
            self.nombre = nombre
        else:
            raise ValueError("Entre 1 a 100 caracteres")

        self.precio = precio
        

        if isinstance(cantidad, int) and cantidad >= 0:
            self.cantidad = cantidad
        else:
            raise ValueError("Cantidad debe ser un número no negativo")

        if isinstance(tipo, str) and 0 <= len(tipo) <= 20:
            self.tipo = tipo
        else:
            raise ValueError("Tipo debe tener entre 0 y 20 caracteres")
    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, value):
        if value >= 10 and value <= 10000:
            self._precio = value

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, value):
        if value >= 0 and value <= 100:
            self._cantidad = value

    def valorTotal(self):
        return self.precio * self.cantidad



In [310]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [311]:
class Oferta:
    def __init__(self, descripcion, codigos=[], tipos=[]):
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos

    def esAplicable(self, producto):
        return producto.codigo in self.codigos or producto.tipo in self.tipos
        raise NotImplementedError()


class Oferta2x1(Oferta):
    def __init__(self, tipos=[]):
        super().__init__(f"Oferta 2x1 en productos de tipo {', '.join(tipos)}", tipos=tipos)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto):
            return (cantidad // 2) * producto.precio
        return 0


class OfertaDescuento(Oferta):
    def __init__(self, porcentaje, codigos=[]):
        super().__init__(f"Descuento del {porcentaje}% en productos con código {', '.join(codigos)}", codigos=codigos)
        self.porcentaje = porcentaje

    def calcularDescuento(self, producto, cantidad):
        if producto.codigo in self.codigos:
            subtotal = producto.precio * cantidad
            return subtotal * (self.porcentaje / 100)
        return 0





In [312]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [313]:
import csv

class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []
        
    def agregar(self, *productos):
        self.productos.extend(productos)
     

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)


    def reducirStock(self, codigo, cantidad):
        for producto in self.productos:
            if producto.codigo == codigo:
                if cantidad > producto.cantidad:
                    raise ValueError('No hay suficiente stock disponible')
                producto.cantidad -= cantidad
                return  # Salir una vez que se ha reducido el stock
        raise ValueError('Producto no encontrado')
    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos)

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        descuento_total = 0
        for oferta in self.ofertas:
            descuento_oferta = oferta.calcularDescuento(producto, cantidad)
            if descuento_oferta > 0:
                descuento_total = max(descuento_total, descuento_oferta)
        return descuento_total

    def vender(self, producto, cantidad):
        if producto.cantidad >= cantidad:
            producto.cantidad -= cantidad
        else:
            raise ValueError("No hay suficiente stock para vender")

    def informe(self):
        informe = []
        informe.append(f"Cantidad de productos: {self.cantidadProductos}")
        informe.append(f"Cantidad de unidades: {self.cantidadUnidades}")
        informe.append(f"Precio Promedio: {self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0}")
        informe.append(f"Valor total: {self.valorTotal}")
        informe.append(f"Tipos de productos: {', '.join(set(p.tipo for p in self.productos))}")
        
        informe.append("Ofertas:")
        for oferta in self.ofertas:
            informe.append(f"- {oferta.descripcion}")
        
        return "\n".join(informe)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def guardar(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])
                print(catalogo.informe())

    @staticmethod
    def leer(filename):
        catalogo = Catalogo()
        try:
            with open(filename, 'r') as csvfile:
                reader = csv.reader(csvfile)
                next(reader)  
                for row in reader:
                    codigo, nombre, precio, tipo, cantidad = row
                    producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                    catalogo.agregar(producto)
            return catalogo
        except FileNotFoundError:
            print(f"El archivo {filename} no se encontró.")
            return None


In [314]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Cantidad de productos: 4
Cantidad de unidades: 100
Precio Promedio: 1670.0
Valor total: 167000
Tipos de productos: gaseosa, galleta
Ofertas:
- Oferta 2x1 en productos de tipo galleta
- Descuento del 10% en productos con código 0001, 0003
Cantidad de productos: 4
Cantidad de unidades: 100
Precio Promedio: 1670.0
Valor total: 167000
Tipos de productos: gaseosa, galleta
Ofertas:
- Oferta 2x1 en productos de tipo galleta
- Descuento del 10% en productos con código 0001, 0003
Cantidad de productos: 4
Cantidad de unidades: 100
Precio Promedio: 1670.0
Valor total: 167000
Tipos de productos: gaseosa, galleta
Ofertas:
- Oferta 2x1 en productos de tipo galleta
- Descuento del 10% en productos con código 0001, 0003
Cantidad de productos: 4
Cantidad de unidades: 100
Precio Promedio: 1670.0
Valor total: 167000
Tipos de productos: gaseosa, galleta
Ofertas:
- Oferta 2x1 en productos de tipo galleta
- Descuento del 10% en productos con código 0001, 0003
Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [315]:
import re

class Cliente:
    def __init__(self, nombre, cuit):
        self._nombre = None
        self._cuit = None
        
        self.nombre = nombre  
        self.cuit = cuit    

    @property
    def nombre(self):
        return self._nombre

    @nombre.setter
    def nombre(self, value):
       
        if 1 <= len(value) <= 100:
            self._nombre = value
        else:
            
            print(f"Nombre inválido: {value}, se mantiene el anterior.")

    @property
    def cuit(self):
        return self._cuit

    @cuit.setter
    def cuit(self, value):
        
        if re.match(r"^\d{2}-\d{8}-\d{1}$", value):
            self._cuit = value
        else:
            
            print(f"CUIT inválido: {value}, se mantiene el anterior.")




In [316]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Nombre inválido: , se mantiene el anterior.
CUIT inválido: 1234567890123, se mantiene el anterior.
CUIT inválido: CC-12345678-1, se mantiene el anterior.
Prueba pasada exitosamente!


In [317]:
@classmethod
def nuevoNumero(cls):
    cls._numeroSecuencial += 1
    return cls._numeroSecuencial

@classmethod
def ultimaFactura(cls, numero):
    cls._numeroSecuencial = numero

def __init__(self, catalogo, cliente):
    self.numero = Factura.nuevoNumero()  
    self.fecha = datetime.now().strftime("%Y-%m-%d")  
    self.cliente = cliente  
    self.catalogo = catalogo  
    self.items = []  
    self.subtotal = 0
    self.descuentos = 0
    self.total = 0

@property
def cantidadProductos(self):
    return len(self.items)

@property
def cantidadUnidades(self):  
    return sum(cantidad for _, cantidad in self.items)

def agregar(self, producto, cantidad):
    for i, (prod, cant) in enumerate(self.items):
        if prod.codigo == producto.codigo:
            self.items[i] = (prod, cant + cantidad)  
            break
    else:
        self.items.append((producto, cantidad))  

    try:
        self.catalogo.reducirStock(producto.codigo, cantidad)
    except ValueError as e:
        print(f'Error al reducir stock: {e}')
        return

    self.calcularTotales()

def calcularTotales(self):
    self.subtotal = 0
    self.descuentos = 0
    for producto, cantidad in self.items:
        precio_total = producto.precio * cantidad
        self.subtotal += precio_total
        
        descuento = self.catalogo.calcularDescuento(producto, cantidad)
        self.descuentos += descuento

    self.total = self.subtotal - self.descuentos

    def imprimir(self ):
        factura = []
        
        factura.append(f"Factura: {self.numero}")
        factura.append(f"Fecha  : {self.fecha}")
        factura.append(f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n")

        productos_agrupados = {}

        for producto, cantidad in self.items:
            if producto.codigo in productos_agrupados:
                productos_agrupados[producto.codigo]['cantidad'] += cantidad
            else:
                productos_agrupados[producto.codigo] = {'producto': producto, 'cantidad': cantidad}

        for datos in productos_agrupados.values():
            producto = datos['producto']
            cantidad = datos['cantidad']
            subtotal_item = producto.precio * cantidad
            factura.append(f"{cantidad} {producto.nombre:<20} x ${producto.precio:.2f} = ${subtotal_item:.2f}")

            descuento_item = self.catalogo.calcularDescuento(producto, cantidad)
            if descuento_item > 0:
                factura.append(f"  Descuento: - ${descuento_item:.2f}")

                for oferta in self.catalogo.ofertas:
                    if oferta.esAplicable(producto):
                        factura.append(f"      Oferta: {oferta.descripcion}")

        factura.append("\n" + " " * 42 + f"Subtotal:   ${self.subtotal:.2f}")
        factura.append(" " * 42 + f"Descuentos: ${self.descuentos:.2f}")
        factura.append(" " * 42 + "-" * 23)
        factura.append(" " * 42 + f"TOTAL:      ${self.total:.2f}")  

        return "\n".join(factura)

# Uso del código
factura = Factura(catalogo, cliente)
print(factura.imprimir())





Factura: 104
Fecha  : 05/10/2024
Cliente: Juan Perez (20-12345678-9)


                                          Subtotal:   $0.00
                                          Descuentos: $0.00
                                          -----------------------
                                          TOTAL:      $0.00


In [318]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

AssertionError: 

In [160]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

TypeError: Oferta2x1.__init__() got an unexpected keyword argument 'codigos'